In [10]:
import os
from typing import Any, TypedDict, List, Dict, Annotated
from langchain_core.messages import BaseMessage
from langchain_core.agents import AgentAction
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.tools import tool

with open("/Users/jkatigbak/.config/openai", "r") as f:
    api_key = f.read().strip()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=api_key)


class StoryParameters(TypedDict):
    age_range: str
    themes: List[str]
    moral: str
    characters: List[Dict]
    story_setting: str
    tone: str
    parental_concerns: List[str]
    inclusion_and_diversity: str
    ending: str

class StoryState(TypedDict):
    user_input: str
    chat_history: List[BaseMessage]
    intermediate_steps: List[AgentAction]
    story_parameters: Dict
    outline: Dict
    enhanced_outline: Dict
    developed_sections: Dict[str, str]
    full_story: str
    scenes: List[Dict]
    images: List[str]
    sender: str


class AnnotatedStoryState(TypedDict):
    user_input: Annotated[str, "User input or system-generated prompt"]
    chat_history: Annotated[List[BaseMessage], "History of conversation between user and AI"]
    intermediate_steps: Annotated[List[AgentAction], "Steps taken by the agent during processing"]
    story_parameters: Annotated[Dict, "Parameters defining the story structure and elements"]
    outline: Annotated[Dict, "Initial story outline"]
    enhanced_outline: Annotated[Dict, "Detailed story outline with additional elements"]
    developed_sections: Annotated[Dict[str, str], "Fully developed narrative sections of the story"]
    full_story: Annotated[str, "Complete narrative of the story"]
    scenes: Annotated[List[Dict], "List of scene descriptions for image generation"]
    images: Annotated[List[str], "List of generated image URLs or references"]
    sender: Annotated[str, "Identifier of the entity sending the current message"]


def get_state_value(state: StoryState | AnnotatedStoryState, key: str) -> Any:
    """
    Helper function to retrieve a value from the StoryState given a key.
    
    Args:
    state (StoryState): The current state of the story.
    key (str): The key to retrieve from the state.
    
    Returns:
    Any: The value associated with the given key in the state.
    
    Raises:
    KeyError: If the key is not found in the state.
    """
    if key in state:
        return state[key]
    else:
        raise KeyError(f"Key '{key}' not found in StoryState")


In [14]:
# default story parameters

story_parameters = {
    "ageRange": "3-5",
    "themes": ["friendship", "courage"],
    "moral": "Be brave and stand up for what's right",
    "characters": [
        {"name": "Timmy", "description": "A shy but brave young boy"},
        {"name": "Sarah", "description": "Timmy's adventurous friend"}
    ],
    "storySetting": "A forest", 
    "tone": "Whimsical and adventurous"
}



In [29]:
class Scene(TypedDict):
    setting: str
    characters: List[Dict]
    text: str
    image_prompt: str
    image_url: str

class SectionState(TypedDict):
    name: str
    outline: str
    scenes: List[Scene]

def create_synopsis(llm: ChatOpenAI, story_parameters: StoryParameters) -> str:
    """
      Given a set of parameters, create a brief synopsis for a children's story.
    """
    prompt = f"""Create a brief synopsis for a children's story based on these parameters:
    Age Range: {story_parameters['ageRange']}
    Themes: {', '.join(story_parameters['themes'])}
    Moral: {story_parameters['moral']}
    Characters: {', '.join([f"{c['name']} ({c['description']})" for c in story_parameters['characters']])}
    Setting: {story_parameters['storySetting']}
    Tone: {story_parameters['tone']}

    The synopsis should be 2-3 sentences long and capture the essence of the story.
    """
    
    response = llm.invoke(prompt)
    return response.content

def generate_outline(llm: ChatOpenAI, story_parameters: StoryParameters) -> str | Dict:
    """
    Given a set of parameters, generate an outline for a children's story.
    """
    prompt = f"""
    # Story Outline Generation Prompt for Children's Storybook

    ### Goal
    Create a structured outline for a children's story based on the provided template. The story should have a fulfilling and cohesive narrative arc, similar to the style of a world-renowned children's author (e.g., Dr. Seuss, Beatrix Potter, Roald Dahl). The outline should be divided into parts, including key plot points, character development, and narrative progression. The outline will serve as a framework for further expansion into a full story.

    ### Instructions
    Using the template provided below, generate an outline for a children's story. The outline should include the following sections:

    1. Introduction: Introduce the main characters, setting, and initial situation.
    2. Conflict/Problem: Describe the main conflict or problem the characters face.
    3. Rising Action: Outline the events and challenges that lead to the climax.
    4. Climax: Highlight the most intense or crucial moment of the story.
    5. Falling Action: Show how the characters begin to resolve the conflict or problem.
    6. Resolution: Conclude the story with a satisfying resolution that ties back to the moral or lesson.

    ### Story Elements to Include:

    - Age Range: {story_parameters['ageRange']}
    - Story Length: {story_parameters.get('storyLength', '800 words')}
    - Themes: {', '.join(story_parameters['themes'])}
    - Moral of the Story: {story_parameters['moral']}
    - Characters: {', '.join([f"{{c['name']}} ({{c['description']}})" for c in story_parameters['characters']])}
    - Story Setting: {story_parameters['storySetting']}
    - Tone: {story_parameters['tone']}
    - Parental Concerns: {', '.join(story_parameters.get('parentalConcerns', ['None specified']))}
    - Inclusion and Diversity: {story_parameters.get('inclusionAndDiversity', 'Include diverse characters and perspectives')}
    - Ending: {story_parameters.get('ending', 'Happy ending with a moral resolution')}
    - Additional Instructions: {story_parameters.get('additionalInstructions', 'None')}

    ### Output Template:

    ```markdown
    # Story Outline

    ## Setting
    - Brief description of the story setting
    - Themes, moral, etc.
    - Style, tone, etc.
    
    ## Character(s)
    - name, description, traits, etc. 
    - ...(repeat for each character)

    ## Outline

    ### Introduction
    - Key point(s)
   
    ### Conflict
    - Key point(s)

    ### Rising Action
    - Key point(s)

    ### Climax
    - Key point(s)

    ### Falling Action
    - Key point(s)

    ### Resolution
    - Key point(s)
    ```

    Generate the output based on the provided parameters.
"""
    
    response = llm.invoke(prompt)
    return response.content

def enhance_outline(llm: ChatOpenAI, outline: Dict):
    """
    Given an outline, enhance it by adding more details and coherence to each section.
    """
    prompt = f"""
# Story Outline Enhancement Prompt for Children's Storybook

## Goal
You are a story enhancement model whose goal is to enhance and refine the initial outline for a children's storybook that gets provided to you, adding depth, detail, and coherence to each section while maintaining overall narrative consistency. This enhanced outline will serve as a more robust framework for the full story development process.

## Core Objectives

1. Expand each section of the initial outline with more specific details and plot points.
2. Ensure strong connections between different parts of the story for a cohesive narrative flow.
3. Develop character arcs and relationships throughout the outline.
4. Integrate the story's themes and moral lessons more deeply into the plot structure.
5. Maintain age-appropriateness and adherence to the specified tone and style.
6. Address any potential plot holes or inconsistencies in the initial outline.

## Output Guidelines

Produce an enhanced outline that:
1. Expands each of the six main sections with 3-5 specific plot points or details.
2. Includes character development notes for main characters in relevant sections.
3. Specifies how themes and moral lessons are demonstrated in each part of the story.
4. Suggests potential story beats or moments that could be good for illustrations.
5. Maintains the overall structure and flow of the original outline while adding depth and nuance.

## Enhancement Guidelines

1. Character Development:
   - Add notes on how each main character grows or changes in different parts of the story.
   - Include specific character interactions or moments that reveal personality traits.

2. Theme Integration:
   - For each section, specify how the story's themes are demonstrated or explored.
   - Ensure the moral lesson is built up gradually throughout the story, not just at the end.

3. Plot Detailing:
   - Break down general events into more specific occurrences or challenges.
   - Add small subplots or side quests that contribute to the main story arc.

4. Setting Elaboration:
   - Include more specific details about the story's setting in each section.
   - Note how the setting might change or be perceived differently as the story progresses.

5. Emotional Journey:
   - Highlight the emotional states of the characters at different points in the story.
   - Ensure there's an emotional arc that aligns with the plot arc.

6. Age-Appropriate Complexity:
   - Adjust the level of detail and complexity to suit the target age range.
   - Ensure challenges and resolutions are understandable and relatable for young readers.

7. Illustration Opportunities:
   - Note potential moments or scenes that would make for engaging illustrations.
   - Consider visual variety in these moments (e.g., action scenes, emotional moments, beautiful settings).

## Consistency Checks

Before finalizing the enhanced outline, verify:

1. Narrative Flow: Does each section logically lead to the next?
2. Character Consistency: Are characters' actions and growth consistent with their established traits?
3. Theme Coherence: Are the story's themes woven consistently throughout the outline?
4. Age Appropriateness: Does the enhanced content remain suitable for the target age range?
5. Pacing: Is there a good balance of action, reflection, and development throughout the story?
6. Setting Continuity: Is the story world presented consistently across all sections?

## Before Outputting

Ensure the enhanced outline:
1. Maintains the core story structure while providing richer detail.
2. Offers clear guidance for the full story development process.
3. Balances plot progression, character development, and theme exploration.
4. Provides ample opportunities for engaging illustrations.
5. Remains true to the original story parameters and intentions.

Remember, your goal is to create a more robust and detailed outline that will serve as a strong foundation for developing a engaging and cohesive children's storybook. 

## Outline

{outline}

### Output:
"""
    response = llm.invoke(prompt)
    return response.content

def develop_section(llm: ChatOpenAI, section_name: str, section_outline: Dict, additional_context: Dict):
    prompt = f"""
    # Story Development Prompt for Children's Storybook

    You are a specialized AI agent designed to take a specified portion of a story outline and fully develop that section into a detailed narrative for a children's storybook. Your goal is to create engaging, age-appropriate content that aligns with the overall story structure and themes.

    ## Core Objectives

    1. Expand a given story outline section into a fully developed narrative.
    2. Maintain consistency with the overall story arc, character development, and thematic elements.
    3. Create engaging, age-appropriate content that captures the imagination of young readers.
    4. Incorporate descriptive language, dialogue, and action to bring the story to life.
    5. Ensure the developed section flows smoothly from and into the surrounding narrative.

    ## Input

    You will receive:
    1. The section of the story outline to be developed (e.g., Introduction, Conflict/Problem, Rising Action, Climax, Falling Action, Resolution)
    2. The complete story outline for context
    3. Character descriptions
    4. Setting details
    5. Overall story information (theme, tone, moral)
    6. Target age range
    7. Any specific storytelling instructions or preferences

    ## Output

    Produce a detailed narrative section that:
    1. Expands on the given outline section, typically 200-400 words depending on the age range and complexity of the story.
    2. Includes character dialogue, actions, and internal thoughts as appropriate.
    3. Describes the setting and atmosphere in vivid, age-appropriate language.
    4. Advances the plot in a way that is consistent with the overall story arc.
    5. Incorporates the story's themes and moral lessons where relevant.

    ## Development Guidelines

    1. Character Development:
    - Ensure characters act and speak consistently with their established personalities.
    - Show character growth or change through their actions and decisions.

    2. Setting and Atmosphere:
    - Use descriptive language to paint a picture of the environment.
    - Incorporate sensory details to make the setting feel alive.

    3. Dialogue:
    - Write age-appropriate dialogue that sounds natural and distinct for each character.
    - Use dialogue to reveal character personalities and advance the plot.

    4. Pacing:
    - Maintain an appropriate pace for the story section you're developing.
    - Use a mix of action, dialogue, and description to keep the narrative engaging.

    5. Theme and Moral:
    - Weave the story's themes and moral lessons into the narrative in a subtle, natural way.
    - Avoid being overly didactic; let the story itself convey the message.

    6. Age Appropriateness:
    - Adjust vocabulary and concept complexity to suit the target age range.
    - Ensure all content is suitable for young readers.

    7. Continuity:
    - Ensure your developed section flows smoothly from the previous part of the story and into the next.
    - Reference earlier events or foreshadow future ones where appropriate.

    ## Consistency Checks

    Before finalizing your developed section, verify:

    1. Character Consistency: Do the characters behave in line with their established traits?
    2. Plot Alignment: Does this section advance the story as outlined in the overall arc?
    3. Thematic Integration: Are the story's themes naturally incorporated?
    4. Age Appropriateness: Is the content suitable for the target age range in vocabulary and concepts?
    5. Pacing: Does the section maintain an engaging pace and flow well with the rest of the story?
    6. Setting Continuity: Is the setting described consistently with earlier descriptions?

    ## Before Outputting

    Ensure the developed section:
    1. Is the appropriate length for the story and age range
    2. Captures the essence of the outline section while adding depth and detail
    3. Engages the reader with vivid descriptions and compelling dialogue
    4. Advances character development and plot in meaningful ways
    5. Aligns with the overall story themes and moral
    6. Flows smoothly as part of the larger narrative

    Remember, your goal is to create a rich, engaging narrative that brings the story outline to life, captivates young readers, and seamlessly fits into the overall storybook.
    """
    response = llm.invoke(prompt)
    return response.content


def paginate_section_text(llm: ChatOpenAI, section_text: str):
    prompt = f"""
    Organize the following text into chunks of sentences that would be used in a children's storybook. Return the result as a list of strings.
    Text:
    {section_text}
    """
    response = llm.invoke(prompt)
    # try to eval the response
    try:
        return eval(response.content)
    except:
        return troubleshoot_output(llm, response.content, "List[str]", "paginate_section_text")

def generate_image_description(llm: ChatOpenAI, scene: Dict, story_parameters: Dict):
    prompt = f"""
    # Image Description Generator for Children's Storybooks

    You are a specialized AI agent designed to create detailed image descriptions for children's storybooks. Your primary goal is to ensure visual consistency, maintain narrative flow, and create engaging, age-appropriate content that can be used as prompts for image generation.

    ## Core Objectives

    1. Create detailed, consistent image descriptions for each page of the storybook.
    2. Ensure consistency in character representation, setting, and overall style throughout the book.
    3. Adapt the complexity and content to the specified age range.
    4. Maintain the story's themes, moral lessons, and emotional tone.
    5. Avoid any copyrighted characters, settings, or specific artistic styles.

    ## Input

    You will receive:
    1. A story fragment
    2. Overall story information (theme, tone, moral)
    3. Character descriptions
    4. Setting details
    5. Target age range
    6. Any specific style or artistic references

    ## Image Description Guidelines

    1. Style: Describe the overall art style and color palette without referencing specific artists or copyrighted works.
    2. Scene: Detail the setting and important environmental elements.
    3. Characters: Describe each character present, including their appearance, pose, and expression, using original descriptions.
    4. Action: Describe the main action or focus of the image.
    5. Perspective: Specify the viewpoint and composition.
    6. Lighting: Describe lighting conditions and any atmospheric effects.
    7. Avoid Copyrighted Content: Ensure all descriptions are original and do not reference specific copyrighted characters, settings, or art styles, if they do rewrite it in a way that would not infringe on copyrights.

    ## Consistency Checks

    Before finalizing each image description, verify:

    1. Character Continuity: Are characters consistently represented in appearance and personality?
    2. Setting Continuity: Does the setting align with previous descriptions and the overall world of the story?
    3. Style Consistency: Does the described art style match the established style for the book without infringing on copyrights?
    4. Age Appropriateness: Is the content suitable for the target age range in subject matter?
    5. Thematic Alignment: Does the image contribute to the overall themes and moral of the story?
    6. Emotional Tone: Does the mood of the image align with the intended emotional journey of the story?

    ## Before Outputting

    Verify the following:

    1. Originality: Ensure the image description doesn't reference any copyrighted material.
    2. Completeness: Is the image description detailed enough for accurate generation?
    3. Consistency: Does this image align with the overall story and previous descriptions?
    4. Engagement: Is the image likely to captivate the target audience?
    5. Clarity: Is the image description clear and unambiguous?
    6. Progression: Does this image effectively move the story forward?

    Remember, your goal is to create vivid, imaginative scene descriptions that bring the story to life for young readers while ensuring all content is original and copyright-free.

    ### Input:
    settings: {scene['setting']}
    characters: {scene['characters']}
    text: {scene['text']}
    age_range: {story_parameters['ageRange']}
    style: {story_parameters['style']}
    tone: {story_parameters['tone']}
    themes: {story_parameters['themes']}

    ### Output:

    """
    # Implementation to generate image descriptions
    response = llm.invoke(prompt)
    return response.content

def generate_image(llm: ChatOpenAI, image_prompt: str):
    from langchain_openai import OpenAIClient
    from langchain_core.messages import HumanMessage
    
    def generate_image(llm: ChatOpenAI, image_prompt: str):
        client = OpenAIClient()
        response = client.images.generate(
            model="dall-e-3",
            prompt=image_prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        
        image_url = response.data[0].url
        return image_url


def troubleshoot_output(llm: ChatOpenAI, output: str, expected_type: str, source_fn: str):
    prompt = f"""
    We tried to eval the following output but got an error, please fix the output to be a valid python object of type {expected_type}.

    Output:
    {output}
    """
    response = llm.invoke(prompt)
    try:
        content = eval(response.content)
        if type(content) == expected_type:
            return content
        else:
            return troubleshoot_output(llm, response.content, expected_type, f"{source_fn}.troubleshoot_output")
    except:
        raise ValueError(f"Output is not a valid {expected_type} in {source_fn}")



In [ ]:
from langgraph.graph import StateGraph, START, END


def section_builder(state: StoryState) -> StateGraph:
    section_builder = StateGraph(StoryState)
    
    section_builder.add_node("DevelopSection", lambda state: develop_section(llm, state['name'], state['outline'], state['additional_context']))
    section_builder.add_node("ExtractScenes", lambda state: paginate_section_text(llm, state['full_story']))
    section_builder.add_node("GenerateImageDescriptions", lambda state: generate_image_description(llm, state['scene']))
    section_builder.add_node("GenerateImages", lambda state: generate_image(llm, state['image_prompt']))
    
    section_builder.add_edge(START, "DevelopSection")
    section_builder.add_edge("DevelopSection", "ExtractScenes")
    section_builder.add_edge("ExtractScenes", "GenerateImageDescriptions")
    section_builder.add_edge("GenerateImageDescriptions", "GenerateImages")
    section_builder.add_edge("GenerateImages", END)

    section_graph = section_builder.compile()
    return section_graph
    

In [24]:
synopsis = (create_synopsis(llm, story_parameters))
outline = (generate_outline(llm, story_parameters))


In [30]:
enhanced_outline = (enhance_outline(llm, outline))
enhanced_outline

"# Enhanced Story Outline\n\n## Setting\n- The story takes place in a colorful and whimsical forest, named **Wonderwood**, filled with towering trees, sparkling streams, and a variety of friendly animals. The forest is vibrant, with flowers of every color, paths that twist and turn, leading to hidden clearings, magical spots, and a soft, inviting carpet of moss underfoot. The sounds of chirping birds and bubbling brooks create a lively atmosphere.\n- **Themes**: Friendship, Courage, Empathy\n- **Moral**: Be brave and stand up for what's right, and understanding others can lead to kindness.\n- **Style**: Whimsical and adventurous, filled with playful language and rhymes that engage young readers, incorporating illustrations of the forest's beauty and the characters' emotions.\n\n## Character(s)\n- **Lila the Lively Rabbit**: A small, fluffy rabbit with big ears, a bright pink nose, and a heart full of courage. She is curious, adventurous, and always ready to help her friends. Throughout

In [39]:
import re

def parse_markdown(markdown_text):
    """
    Parse markdown text and extract structured information.
    
    Args:
    markdown_text (str): The markdown text to parse.
    
    Returns:
    dict: A dictionary containing parsed sections and their content.
    """
    sections = {}
    current_section = None
    current_content = []

    for line in markdown_text.split('\n'):
        # Check for headers
        header_match = re.match(r'^(#+)\s+(.+)$', line)
        if header_match:
            if current_section:
                sections[current_section] = '\n'.join(current_content).strip()
            level = len(header_match.group(1))
            current_section = (level, header_match.group(2))
            current_content = []
        else:
            current_content.append(line)

    # Add the last section
    if current_section:
        sections[current_section] = '\n'.join(current_content).strip()

    return sections

parsed_outline = parse_markdown(enhanced_outline)
parsed_outline

def create_structured_outline(parsed_outline):
    def split_section(section_content):
        parts = section_content.split('\n')
        return [part.strip('- ') for part in parts if part.strip()]

    structured_outline = {
        "title": "Enhanced Story Outline",
        "setting": split_section(parsed_outline[(2, "Setting")]),
        "characters": split_section(parsed_outline[(2, "Character(s)")]),
        "themes": ["Friendship", "Courage", "Empathy"],
        "moral": "Be brave and stand up for what's right, and understanding others can lead to kindness.",
        "style": "Whimsical and adventurous, filled with playful language and rhymes that engage young readers, incorporating illustrations of the forest's beauty and the characters' emotions.",
        "plot": {
            section: split_section(parsed_outline[(3, section)])
            for section in ["Introduction", "Conflict", "Rising Action", "Climax", "Falling Action", "Resolution"]
        }
    }
    return structured_outline

structured_outline = create_structured_outline(parsed_outline)

# Print the structured outline
print(structured_outline)




{'title': 'Enhanced Story Outline', 'setting': ['The story takes place in a colorful and whimsical forest, named **Wonderwood**, filled with towering trees, sparkling streams, and a variety of friendly animals. The forest is vibrant, with flowers of every color, paths that twist and turn, leading to hidden clearings, magical spots, and a soft, inviting carpet of moss underfoot. The sounds of chirping birds and bubbling brooks create a lively atmosphere.', '**Themes**: Friendship, Courage, Empathy', "**Moral**: Be brave and stand up for what's right, and understanding others can lead to kindness.", "**Style**: Whimsical and adventurous, filled with playful language and rhymes that engage young readers, incorporating illustrations of the forest's beauty and the characters' emotions."], 'characters': ['**Lila the Lively Rabbit**: A small, fluffy rabbit with big ears, a bright pink nose, and a heart full of courage. She is curious, adventurous, and always ready to help her friends. Through

NameError: name 'StoryState' is not defined